In this script, we demonstrate how to initialize and change values in TFP.

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors


Let's set up a Beta prior/posterior so that we have a clearly constrained posterior support of (0, 1).

In [25]:
default_fit = tfp.experimental.vi.build_factored_surrogate_posterior(
    event_shape=(),
    bijector=tfb.Sigmoid(),
)

init_fit = tfp.experimental.vi.build_factored_surrogate_posterior(
    event_shape=(),
    bijector=tfb.Sigmoid(),
    initial_parameters={
        'loc': tf.Variable(1.0),
        'scale': tf.Variable(0.8),
    }
)

Let's examine whether the initialized values are in the constrained or unconstrained space.

In [35]:
default_initial_values = default_fit.distribution.loc.numpy(), default_fit.distribution.scale.numpy()
init_initial_values = init_fit.distribution.loc.numpy(), init_fit.distribution.scale.numpy()
print(f"The initialized values by default are {default_initial_values[0]}, {default_initial_values[1]}")
print(f"The initialized values when we set loc=1.0 and scale=0.1 are {init_initial_values[0]}, {init_initial_values[1]}")

The initialized values by default are -0.23432374000549316, 0.009999997913837433
The initialized values when we set loc=1.0 and scale=0.1 are 1.0, 0.8000000715255737


In [36]:
# empirically observe the mean and std of the samples
init_samples = init_fit.sample(100_000)
default_samples = default_fit.sample(100_000)
print(f"Default fit (constrained) empirical mean and std: {np.mean(default_samples.numpy())}, {np.std(default_samples.numpy())}")
print(f"Init fit (constrained) empirical mean and std: {np.mean(init_samples.numpy())}, {np.std(init_samples.numpy())}")

Default fit (constrained) empirical mean and std: 0.44170063734054565, 0.0024616378359496593
Init fit (constrained) empirical mean and std: 0.7067192792892456, 0.15060171484947205


It seems that the values we observe empirically are different from the initialized values, so let's transform the initialized values and see if they match the empirical values. (we use measure transport, i.e. we draw samples from the unconstrained distribution, transform the samples, and then take the empirical mean and std of the transformed samples)

In [37]:
# sample from the initialized values and transform to constrained space to find empirical constrained mean and std from
# initial values
init_constrained_samples = tfb.Sigmoid().forward(
    tfd.Normal(loc=init_fit.distribution.loc, scale=init_fit.distribution.scale).sample(100_000)
)
default_constrained_samples = tfb.Sigmoid().forward(
    tfd.Normal(loc=default_fit.distribution.loc, scale=default_fit.distribution.scale).sample(100_000)
)
print(f"Default fit empirical constrained mean and std from initial values: {np.mean(default_constrained_samples.numpy())}, {np.std(default_constrained_samples.numpy())}")
print(f"Init fit empirical constrained mean and std from initial values: {np.mean(init_constrained_samples.numpy())}, {np.std(init_constrained_samples.numpy())}")

Default fit empirical constrained mean and std from initial values: 0.4417007863521576, 0.002471410669386387
Init fit empirical constrained mean and std from initial values: 0.7071577906608582, 0.15019743144512177


Since these values match the constrained mean and std above, we conclude that TFP's initialization control changes the underlying unconstrained gaussian distribution in ADVI.